In [25]:
import numpy as np
import nibabel as nib
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import pandas as pd
from niipreprocess11 import  niipreprocess as npp
#from niiaimodel import niimodelbuild as nmb
#from pandas_profiling import ProfileReport
import os
import glob
import warnings
from sklearn import metrics
warnings.filterwarnings("ignore")

In [26]:
files = glob.glob("C:/Users/c0028603/OneDrive - Sheffield Hallam University/05. Dissertation/BRaTS Dataset/BraTSReg_gray_matter_csv_v2/*.csv")
df = pd.DataFrame()
for f in files:
    #if index <=1:
    csv = pd.read_csv(f)
    df = pd.concat([df,csv])
b_dataset = df

In [27]:
b_dataset['istumor'].value_counts()

0    1726731
1       2522
Name: istumor, dtype: int64

In [4]:
#test_dataset['istumor'].value_counts().plot.bar()
from sklearn.utils import resample
#create two different dataframe of majority and minority class 
df_majority = b_dataset[(b_dataset['istumor']==0)] 
df_minority = b_dataset[(b_dataset['istumor']==1)] 
# upsample minority class
df_minority_upsampled = resample(df_minority, 
                                 replace=True,    # sample with replacement
                                 n_samples= len(df_majority), # to match majority class
                                 random_state=42)  # reproducible results
# Combine majority class with upsampled minority class
df_upsampled = pd.concat([df_minority_upsampled, df_majority])

In [5]:
print("coutn of class {}".format(df_upsampled['istumor'].value_counts()))


coutn of class 1    1726731
0    1726731
Name: istumor, dtype: int64


In [28]:
pred_df = pd.read_csv('C:/Users/c0028603/OneDrive - Sheffield Hallam University/05. Dissertation/BRaTS Dataset/BraTSReg_Validation_Data_csv/BraTSReg_142_01_0154_t1ce_values.csv')
svm_pred_df = pd.read_csv('C:/Users/c0028603/OneDrive - Sheffield Hallam University/05. Dissertation/BRaTS Dataset/BraTSReg_gray_matter_csv_v2/BraTSReg_010_01_0083_t1ce.nii.gz_gray_matter.csv')

In [29]:
# Import train_test_split function
from sklearn.model_selection import train_test_split

#X=df_upsampled[['X','pY','VoxVal','Z']]
#y=df_upsampled[['istumor']]
#b_dataset = b_dataset.head( 726731)
print(len(b_dataset))
X=b_dataset[['X','pY','VoxVal','Z']]
y=b_dataset[['istumor']]
# Split dataset into training set and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3) # 70% training and 30% test

1729253


In [12]:
#svm not responding
from sklearn import svm
# fit the model
svm_clf = svm.NuSVC(gamma="auto",nu=0.001)
svm_clf.fit(X_train,y_train)
y_pred=svm_clf.predict(X_test)
# svm model accuracy
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))



Accuracy: 0.9986059968437664


In [13]:
svm_pred_df['istumor_Pred'] = svm_clf.predict(svm_pred_df[['X','pY','VoxVal','Z']])
svm_pred_df.query('istumor == 1').sort_values(['Z'],ascending=True)

,X,pY,VoxVal,Z,NiiFileName,LandmarksFileName,istumor,istumor_Pred
0,87,94,1533.0,34,BraTSReg_010_01_0083_t1ce.nii.gz,BraTSReg_010_01_0083_landmarks.csv,1,1
1,105,101,1547.0,45,BraTSReg_010_01_0083_t1ce.nii.gz,BraTSReg_010_01_0083_landmarks.csv,1,1
2,105,155,857.0,62,BraTSReg_010_01_0083_t1ce.nii.gz,BraTSReg_010_01_0083_landmarks.csv,1,1
3,86,135,1009.0,68,BraTSReg_010_01_0083_t1ce.nii.gz,BraTSReg_010_01_0083_landmarks.csv,1,0
4,128,112,1628.0,73,BraTSReg_010_01_0083_t1ce.nii.gz,BraTSReg_010_01_0083_landmarks.csv,1,0
5,118,97,875.0,92,BraTSReg_010_01_0083_t1ce.nii.gz,BraTSReg_010_01_0083_landmarks.csv,1,0


In [30]:
#Import Random Forest Model
from sklearn.ensemble import RandomForestClassifier
rf_clf=RandomForestClassifier(n_estimators=100)
rf_clf.fit(X_train,y_train)
y_pred=rf_clf.predict(X_test)
# rf model accuracy
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

Accuracy: 0.9985581445556464


In [31]:
svm_pred_df['istumor_Pred'] = rf_clf.predict(svm_pred_df[['X','pY','VoxVal','Z']])
svm_pred_df.query('istumor == 1').sort_values(['Z'],ascending=True)

,X,pY,VoxVal,Z,NiiFileName,LandmarksFileName,istumor,istumor_Pred
0,87,94,1533.0,34,BraTSReg_010_01_0083_t1ce.nii.gz,BraTSReg_010_01_0083_landmarks.csv,1,1
1,105,101,1547.0,45,BraTSReg_010_01_0083_t1ce.nii.gz,BraTSReg_010_01_0083_landmarks.csv,1,1
2,105,155,857.0,62,BraTSReg_010_01_0083_t1ce.nii.gz,BraTSReg_010_01_0083_landmarks.csv,1,1
3,86,135,1009.0,68,BraTSReg_010_01_0083_t1ce.nii.gz,BraTSReg_010_01_0083_landmarks.csv,1,1
4,128,112,1628.0,73,BraTSReg_010_01_0083_t1ce.nii.gz,BraTSReg_010_01_0083_landmarks.csv,1,1
5,118,97,875.0,92,BraTSReg_010_01_0083_t1ce.nii.gz,BraTSReg_010_01_0083_landmarks.csv,1,1


In [33]:
pred_df['istumor_Pred'] = rf_clf.predict(pred_df[['X','pY','VoxVal','Z']])
pred_df.query('istumor_Pred == 1').sort_values(['Z'],ascending=True)

,Unnamed: 0,X,pY,VoxVal,Z,NiiFileName,LandmarksFileName,istumor,istumor_Pred
661268,50736,96,211,2515.424072,68,BraTSReg_142_01_0154_t1ce.nii.gz,BraTSReg_142_01_0154_landmarks.csv,0,1
826370,50766,126,211,3580.228027,77,BraTSReg_142_01_0154_t1ce.nii.gz,BraTSReg_142_01_0154_landmarks.csv,0,1
917658,50766,126,211,2701.193848,82,BraTSReg_142_01_0154_t1ce.nii.gz,BraTSReg_142_01_0154_landmarks.csv,0,1
938661,18576,96,77,2912.270752,84,BraTSReg_142_01_0154_t1ce.nii.gz,BraTSReg_142_01_0154_landmarks.csv,0,1
1045343,15913,73,66,1281.943604,90,BraTSReg_142_01_0154_t1ce.nii.gz,BraTSReg_142_01_0154_landmarks.csv,0,1
1045527,16393,73,68,1346.456055,90,BraTSReg_142_01_0154_t1ce.nii.gz,BraTSReg_142_01_0154_landmarks.csv,0,1
1045621,16633,73,69,1259.177124,90,BraTSReg_142_01_0154_t1ce.nii.gz,BraTSReg_142_01_0154_landmarks.csv,0,1
1148070,47361,81,197,1454.310303,95,BraTSReg_142_01_0154_t1ce.nii.gz,BraTSReg_142_01_0154_landmarks.csv,0,1
1200782,15515,155,64,1296.225830,99,BraTSReg_142_01_0154_t1ce.nii.gz,BraTSReg_142_01_0154_landmarks.csv,0,1
1512043,21990,150,91,1305.942261,122,BraTSReg_142_01_0154_t1ce.nii.gz,BraTSReg_142_01_0154_landmarks.csv,0,1


In [34]:
b_pred_df = pd.read_csv('C:/Users/c0028603/OneDrive - Sheffield Hallam University/05. Dissertation/BRaTS Dataset/BraTSReg_Validation_Data_csv/BraTSReg_141_00_0000_t1ce.nii.gz.csv')
b_pred_df['istumor_Pred'] = rf_clf.predict(b_pred_df[['X','pY','VoxVal','Z']])
b_pred_df.query('istumor_Pred == 1').sort_values(['Z'],ascending=True)

,Unnamed: 0,X,pY,VoxVal,Z,NiiFileName,Y,istumor_Pred
13235,31808,128,132,1076.0,11,BraTSReg_141_00_0000_t1ce.nii.gz,-107,1
23868,31088,128,129,1092.0,14,BraTSReg_141_00_0000_t1ce.nii.gz,-110,1
23953,31808,128,132,1373.0,14,BraTSReg_141_00_0000_t1ce.nii.gz,-107,1
23993,32048,128,133,1411.0,14,BraTSReg_141_00_0000_t1ce.nii.gz,-106,1
24037,32287,127,134,1547.0,14,BraTSReg_141_00_0000_t1ce.nii.gz,-105,1
24038,32288,128,134,1560.0,14,BraTSReg_141_00_0000_t1ce.nii.gz,-105,1
24039,32289,129,134,1332.0,14,BraTSReg_141_00_0000_t1ce.nii.gz,-105,1
24045,32296,136,134,1197.0,14,BraTSReg_141_00_0000_t1ce.nii.gz,-105,1
24046,32297,137,134,1348.0,14,BraTSReg_141_00_0000_t1ce.nii.gz,-105,1
24050,32301,141,134,1157.0,14,BraTSReg_141_00_0000_t1ce.nii.gz,-105,1


In [16]:
# define the keras model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
model = Sequential()
model.add(Dense(12, input_shape=(4,),kernel_initializer = 'uniform', activation='relu'))
model.add(Dense(8,kernel_initializer = 'uniform', activation='relu'))
model.add(Dense(1,kernel_initializer = 'uniform', activation='sigmoid'))
# compile the keras model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
# fit the keras model on the dataset
epoch_history = model.fit(X_train,y_train, epochs=50, batch_size=500000,validation_split=0.2)

Epoch 1/50
1/1 [==============================] - 1s 1s/step - loss: 0.7056 - accuracy: 0.0038 - val_loss: 0.6938 - val_accuracy: 0.4955
Epoch 2/50
1/1 [==============================] - 0s 73ms/step - loss: 0.6938 - accuracy: 0.4935 - val_loss: 0.6812 - val_accuracy: 0.9949
Epoch 3/50
1/1 [==============================] - 0s 67ms/step - loss: 0.6813 - accuracy: 0.9950 - val_loss: 0.6682 - val_accuracy: 0.9990
Epoch 4/50
1/1 [==============================] - 0s 67ms/step - loss: 0.6683 - accuracy: 0.9984 - val_loss: 0.6547 - val_accuracy: 0.9990
Epoch 5/50
1/1 [==============================] - 0s 62ms/step - loss: 0.6548 - accuracy: 0.9984 - val_loss: 0.6407 - val_accuracy: 0.9990
Epoch 6/50
1/1 [==============================] - 0s 64ms/step - loss: 0.6409 - accuracy: 0.9984 - val_loss: 0.6262 - val_accuracy: 0.9990
Epoch 7/50
1/1 [==============================] - 0s 59ms/step - loss: 0.6264 - accuracy: 0.9984 - val_loss: 0.6112 - val_accuracy: 0.9990
Epoch 8/50
1/1 [=============

In [17]:

svm_pred_df['istumor_Pred'] = model.predict(svm_pred_df[['X','pY','VoxVal','Z']],batch_size = 32)
svm_pred_df.query('istumor == 1').sort_values(['Z'],ascending=True)

136/136 [==============================] - 0s 2ms/step


,X,pY,VoxVal,Z,NiiFileName,LandmarksFileName,istumor,istumor_Pred
0,87,94,1533.0,34,BraTSReg_010_01_0083_t1ce.nii.gz,BraTSReg_010_01_0083_landmarks.csv,1,0.000073
1,105,101,1547.0,45,BraTSReg_010_01_0083_t1ce.nii.gz,BraTSReg_010_01_0083_landmarks.csv,1,0.000057
2,105,155,857.0,62,BraTSReg_010_01_0083_t1ce.nii.gz,BraTSReg_010_01_0083_landmarks.csv,1,0.001720
3,86,135,1009.0,68,BraTSReg_010_01_0083_t1ce.nii.gz,BraTSReg_010_01_0083_landmarks.csv,1,0.000898
4,128,112,1628.0,73,BraTSReg_010_01_0083_t1ce.nii.gz,BraTSReg_010_01_0083_landmarks.csv,1,0.000027
5,118,97,875.0,92,BraTSReg_010_01_0083_t1ce.nii.gz,BraTSReg_010_01_0083_landmarks.csv,1,0.001788
